## Movie Data Project

My objective is to leverage Trakt and Boxofficemojo.com's APIs to pull and perform analysis on movie data.

More information on the sources I'm using to collect this data: <br>
Trakt: https://trakt.docs.apiary.io (general movie data) <br>
Boxofficemojo: https://www.boxofficemojo.com (movie revenue data)

The rest of automation would have been done through either Window's Task Scheduler or Jupyter Notebook's Scheduler. <br>
Jupyter Notebook's Scheduler: https://docs.qubole.com/en/latest/user-guide/notebooks-and-dashboards/notebooks/jupyter-notebooks/scheduling-jupy-notebooks.html

In [116]:
# Importing all libraries I could *potentially* use here
import pandas as pd
import re
import requests
import regex
import time
import json
import pickle
import pyodbc
from bs4 import BeautifulSoup

This first part is a web scraper to collect Movie metadata information on Trakt. Only the scrape should be required - I'll be doing the JSON loading and analysis in SQL.

In [117]:
# Create list of years I'd like to pull
years = ['2015','2016','2017','2018','2019']

#Create blank list to drop combine JSONs into
output_list = []

#Necessary headers to make a call to Trakt's API
headers = {'Content-type':'application/json',
           'trakt-api-key':'fb05749dcceb9b63f4c63e3a8b90127ed6e4d56ddb184107b96dc3541cf2b902',
           'trakt-api-version':'2'}
 
#Going to loop through the 5 years I used earlier
for year in years:
    #And this is so I can use pagination to collect more data
    for i in range(1, 11):
        # Create URL
        url = ('https://api.trakt.tv/movies/watched/yearly?years={}&extended=full&page={}&limit=100'.format(year, i))
        
        # Generate request
        res = requests.get(url, headers=headers)
    
        # Change type from bytes to list
        new_list = res.json()
        
        # First run just needs to fill the output_list with initial results
        if year == '2015' and i == 1:
            output_list = new_list
        
        # Subsquent iterations need to add to it
        else:
            output_list += new_list

In [119]:
json_save = json.dumps(output_list)
save = open("movie_data.json", "w")
save.write(json_save)
save.close()

I'll need to stich together revenue data from a different source, as it doesn't to seem to be available (historically at least) via Trakt. This is where Boxofficemojo comes in.

Another web scraper, but this time using BeautifulSoup to parse out the file, then doing some basic data cleanup in python, then saving as an .csv for upload and analysis in SQL.

In [120]:
# Create DF and years list
df_movie_rev = pd.DataFrame([], columns=['movie_title', 'gross', 'total_gross', 'theaters'])
years = ['2015','2016','2017','2018','2019']

# Begin scrape
for year in years:
    url = ('https://www.boxofficemojo.com/yearly/chart/?view2=worldwide&yr={}&p=.htm'.format(year))
    
    # Generate request
    res = requests.get(url)
    
    # Instantiate BeautifulSoup
    soup = BeautifulSoup(res.content, 'html.parser')
    
    # Pull out movie title
    movie_title = soup.find_all(href=re.compile("/release/"))
    
    # Pull out both types of revenue
    # There's no uniqueness in the html to differentiate b/w gross and total gross, so we'll have to seperate out later
    gross_revenue = soup.find_all('td', {'class':'a-text-right mojo-field-type-money mojo-estimatable'})
    
    # Pull out theaters count
    theaters = soup.find_all('td', {'class':'a-text-right mojo-field-type-positive_integer'})
                                
    # Going to take the highest 250 grossing movies from each of the years
    for a in range(0, 250):
        # First iteration's math should work out differently then subsquent runs
        if a == 0:
            df_movie_rev = df_movie_rev.append({'movie_title':movie_title[a].text,
                                                'gross':gross_revenue[a].text,
                                                'total_gross':gross_revenue[a + 1].text, 
                                                'theaters':theaters[a].text}, ignore_index = True)
            
        # Adding in some calcs to variable to make sure the numbers line up                       
        elif a != 0:
            df_movie_rev = df_movie_rev.append({'movie_title':movie_title[a].text,
                                                'gross':gross_revenue[2 * a].text,
                                                'total_gross':gross_revenue[2 * a + 1].text, 
                                                'theaters':theaters[a].text}, ignore_index = True)

In [121]:
# Just to take a look at the DF
df_movie_rev

,movie_title,gross,total_gross,theaters
0,Jurassic World,"$652,270,625","$652,270,625","4,291"
1,Star Wars: Episode VII - The Force Awakens,"$651,967,269","$936,662,225","4,134"
2,Avengers: Age of Ultron,"$459,005,868","$459,005,868","4,276"
3,Inside Out,"$356,461,711","$356,461,711","4,158"
4,Furious 7,"$353,007,020","$353,007,020","4,022"
...,...,...,...,...
1245,Nothing to Lose 2,"$1,150,551","$1,150,551",105
1246,Western Stars,"$1,141,681","$1,581,681",553
1247,Konosuba!: God's Blessing on This Wonderful Wo...,"$1,134,786","$1,134,786",692
1248,My Neighbor Totoro,"$1,105,564","$1,105,564",767


In [122]:
# Basic clean-up of some characters so they don't cause downstream issues
df_movie_rev['movie_title'] = df_movie_rev['movie_title'].map(lambda x: x.replace('"',''))
df_movie_rev['movie_title'] = df_movie_rev['movie_title'].map(lambda x: x.replace("'",''))
df_movie_rev['movie_title'] = df_movie_rev['movie_title'].map(lambda x: x.replace(',',''))
df_movie_rev['gross'] = df_movie_rev['gross'].map(lambda x: x.replace('$',''))
df_movie_rev['gross'] = df_movie_rev['gross'].map(lambda x: x.replace(',',''))
df_movie_rev['total_gross'] = df_movie_rev['total_gross'].map(lambda x: x.replace('$',''))
df_movie_rev['total_gross'] = df_movie_rev['total_gross'].map(lambda x: x.replace(',',''))
df_movie_rev['theaters'] = df_movie_rev['theaters'].map(lambda x: x.replace('$',''))
df_movie_rev['theaters'] = df_movie_rev['theaters'].map(lambda x: x.replace(',',''))

In [123]:
# Dropping into csv
df_movie_rev.to_csv(r"C:\Users\Owner\Documents\Movie_project\movie_rev.csv", index=False)

Now I'm going to step over to SQL to load the data and then run queries for analysis.

In [124]:
#Connecting to the db and creating cursor item
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=WIN8-PC\SQLEXPRESS;DATABASE=master;trusted_connection=yes')
#Leaving this here in case I want to change authentication user later:
#;UID=Win8-PC\Owner;PWD=''')
cursor = conn.cursor()

In [125]:
# Create Schema
cmd_exec_proc = """EXEC Movies.dbo.Setup_Tables"""
conn.autocommit=True
cursor.execute(cmd_exec_proc)

In [126]:
# Drop data into Movies and Revenue tables
cmd_exec_proc = """EXEC Movies.dbo.Raw_data_upload"""
conn.autocommit=True
cursor.execute(cmd_exec_proc)

In [127]:
# Some cleaning and ID assigning for ease of query writing 
cmd_exec_proc = """EXEC Movies.dbo.Data_clean"""
conn.autocommit=True
cursor.execute(cmd_exec_proc)

In [128]:
# Generate reports
cmd_exec_proc = """EXEC Movies.dbo.Run_reports"""
conn.autocommit=True
cursor.execute(cmd_exec_proc)

In [129]:
# Close out cursor object
conn.close()

I'm working on a free version of Microsoft SQL Server, so I don't believe the function msdb.dbo.sp_send_dbmail is available for usage. This step would merely send the data in an email and attached text form. As a result, this data would have to directly viewed in SQL Server for the time being.